In [1]:
from ultralytics import YOLO

# Load a model
# model = YOLO("yolo11n-pose.pt")  # load an official model
model = YOLO(r"E:\Optimizing Workplace Compliance and Safety\models\runs\detect\train4\weights\best.pt")  # load a custom model

# Predict with the model
results = model.track(source = r"E:\project_data\Videos\video3.mp4", show = True, save = True)  # predict on an image

requirements: Ultralytics requirement ['lapx>=0.5.2'] not found, attempting AutoUpdate...
   ---------------------------------------- 1.5/1.5 MB 1.8 MB/s eta 0:00:00

requirements: AutoUpdate success  16.1s, installed 1 package: ['lapx>=0.5.2']
requirements:  Restart runtime or rerun command for updates to take effect

WARNING  Environment does not support cv2.imshow() or PIL Image.show()
OpenCV(4.10.0) D:\a\opencv-python\opencv-python\opencv\modules\highgui\src\window.cpp:1301: error: (-2:Unspecified error) The function is not implemented. Rebuild the library with Windows, GTK+ 2.x or Cocoa support. If you are on Ubuntu or Debian, install libgtk2.0-dev and pkg-config, then re-run cmake or configure script in function 'cvShowImage'



WARNING  inference results will accumulate in RAM unless `stream=True` is passed, causing potential out-of-memory
errors for large sources or long-running streams and videos. See https://docs.ultralytics.com/modes/predict/ for help.

Example:
    results 

In [3]:
import cv2
import numpy as np
from ultralytics import YOLO

# Load YOLOv8 pose estimation model
model = YOLO("yolov8n-pose.pt")

# Video input and output paths
video_path = r"E:\project_data\Videos\kids_running.mp4"
output_path = r"E:\Optimizing Workplace Compliance and Safety\models\runs\detect\track\output.mp4"

# Open video file
cap = cv2.VideoCapture(video_path)

# Get video properties
frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = int(cap.get(cv2.CAP_PROP_FPS))

# Define video writer
fourcc = cv2.VideoWriter_fourcc(*'mp4v')  # Use 'mp4v' for MP4 format
out = cv2.VideoWriter(output_path, fourcc, fps, (frame_width, frame_height))

# Function to calculate angle between three points
def calculate_angle(a, b, c):
    """Calculate the angle between three points (a, b, c) where b is the vertex."""
    ba = np.array(a) - np.array(b)
    bc = np.array(c) - np.array(b)
    cosine_angle = np.dot(ba, bc) / (np.linalg.norm(ba) * np.linalg.norm(bc))
    angle = np.arccos(np.clip(cosine_angle, -1.0, 1.0))  # Clip to avoid floating-point errors
    return np.degrees(angle)

# Function to classify arm-raising based on angles
def classify_arm_raising(keypoints):
    """Detect if a person is raising their arms based on shoulder and elbow angles."""
    if keypoints is None or len(keypoints) < 17:
        return "Unknown"

    left_shoulder, right_shoulder = keypoints[5], keypoints[6]
    left_elbow, right_elbow = keypoints[7], keypoints[8]
    left_wrist, right_wrist = keypoints[9], keypoints[10]

    left_arm_angle = calculate_angle(left_shoulder, left_elbow, left_wrist)
    right_arm_angle = calculate_angle(right_shoulder, right_elbow, right_wrist)

    arm_raised_threshold = 45  # If angle is less than 45°, the arm is raised

    if left_arm_angle < arm_raised_threshold and right_arm_angle < arm_raised_threshold:
        return "Both Arms Raised"
    elif left_arm_angle < arm_raised_threshold:
        return "Left Arm Raised"
    elif right_arm_angle < arm_raised_threshold:
        return "Right Arm Raised"

    return "Arms Down"

# Process video
while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # Run YOLOv8 pose estimation
    results = model(frame)

    for result in results:
        if hasattr(result, "keypoints") and result.keypoints is not None:
            keypoints = result.keypoints.xy.numpy()

            for kp in keypoints:
                if len(kp) < 17:
                    continue

                arm_status = classify_arm_raising(kp)

                # Draw keypoints
                for x, y in kp:
                    cv2.circle(frame, (int(x), int(y)), 5, (0, 255, 0), -1)

                # Display detected arm status
                cv2.putText(frame, arm_status, (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)

    # Write processed frame to output video
    out.write(frame)

    # Show the annotated frame (REMOVE THIS IF RUNNING ON SERVER)
    # cv2.imshow("Arm Raising Detection", frame)  # ❌ REMOVE THIS LINE

    # Add delay for keyboard interrupt handling (optional)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release resources
cap.release()
out.release()
cv2.destroyAllWindows()

print(f"Output video saved as: {output_path}")


0: 384x640 (no detections), 97.0ms
Speed: 5.1ms preprocess, 97.0ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)


error: OpenCV(4.11.0) D:\a\opencv-python\opencv-python\opencv\modules\highgui\src\window.cpp:1367: error: (-2:Unspecified error) The function is not implemented. Rebuild the library with Windows, GTK+ 2.x or Cocoa support. If you are on Ubuntu or Debian, install libgtk2.0-dev and pkg-config, then re-run cmake or configure script in function 'cvWaitKey'


In [ ]:
import cv2
import numpy as np
from ultralytics import YOLO

# Load YOLOv8 pose estimation model
model = YOLO("yolov8n-pose.pt")  # Ensure the model file is available

# Open video file or webcam
video_path = "input_video.mp4"  # Change to 0 for webcam
cap = cv2.VideoCapture(video_path)

# Get video properties
frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = int(cap.get(cv2.CAP_PROP_FPS))

# Define output video writer
output_path = "output_video.avi"
fourcc = cv2.VideoWriter_fourcc(*'XVID')
out = cv2.VideoWriter(output_path, fourcc, fps, (frame_width, frame_height))

# Function to check if the person is touching their face
def is_touching_face(keypoints):
    """
    Detects if a person is touching their face based on hand and face keypoints.
    :param keypoints: Array of detected keypoints
    :return: "Touching Face" or "Not Touching Face"
    """
    if keypoints is None or len(keypoints) < 17:
        return "Unknown"

    # Extract keypoints
    nose = keypoints[0]
    left_eye, right_eye = keypoints[1], keypoints[2]
    left_ear, right_ear = keypoints[3], keypoints[4]
    left_wrist, right_wrist = keypoints[9], keypoints[10]

    # Calculate Euclidean distance function
    def distance(pt1, pt2):
        return np.linalg.norm(np.array(pt1) - np.array(pt2))

    # Define a threshold for face touching (10% of frame height)
    touch_threshold = 0.1 * frame_height

    # Check if the wrist is near any face keypoints
    if (distance(left_wrist, nose) < touch_threshold or
        distance(left_wrist, left_eye) < touch_threshold or
        distance(left_wrist, left_ear) < touch_threshold or
        distance(right_wrist, nose) < touch_threshold or
        distance(right_wrist, right_eye) < touch_threshold or
        distance(right_wrist, right_ear) < touch_threshold):
        return "Touching Face"

    return "Not Touching Face"

# Process video
while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # Run YOLOv8 pose estimation
    results = model(frame)

    for result in results:
        if hasattr(result, "keypoints") and result.keypoints is not None:
            keypoints = result.keypoints.xy.cpu().numpy()  # Extract keypoints

            for kp in keypoints:
                posture = is_touching_face(kp)

                # Draw keypoints
                for x, y in kp:
                    cv2.circle(frame, (int(x), int(y)), 5, (0, 255, 0), -1)

                # Display detected posture
                cv2.putText(frame, posture, (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)

    # Write processed frame to output video
    out.write(frame)

    # Show the annotated frame
    cv2.imshow("Face Touching Detection", frame)

    # Press 'q' to exit
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release resources
cap.release()
out.release()
cv2.destroyAllWindows()

print(f"Output video saved as: {output_path}")

In [ ]:
import cv2
import numpy as np
from ultralytics import YOLO

# Load YOLOv11 model for pose estimation
model = YOLO("yolo11n-pose.pt")  # Ensure this model supports pose estimation

# Open input video
input_video = "/content/4761432-uhd_2160_4096_25fps.mp4"  # Change this to your video file path
cap = cv2.VideoCapture(input_video)

# Check if video file is opened successfully
if not cap.isOpened():
    print("Error: Could not open video file.")
    exit()

# Get video properties
frame_width = int(cap.get(3))
frame_height = int(cap.get(4))
fps = int(cap.get(cv2.CAP_PROP_FPS))

# Define output video writer
output_path = "/content/output_jumping4.mp4"  # Set output path
fourcc = cv2.VideoWriter_fourcc(*"mp4v")
video_writer = cv2.VideoWriter(output_path, fourcc, fps, (frame_width, frame_height))

# Store previous ankle positions to detect jumps
previous_ankle_heights = {}

# Define threshold for detecting jump (relative movement)
jump_threshold = 3  # Adjust based on testing

# Process video frames
while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # Run YOLO pose estimation
    results = model(frame)

    for result in results:
        keypoints = result.keypoints.xy.cpu().numpy()  # Get keypoints

        for person_id, kp in enumerate(keypoints):
            if len(kp) >= 17:  # Ensure all keypoints are available
                left_ankle, right_ankle = kp[15], kp[16]
                left_knee, right_knee = kp[13], kp[14]
                left_hip, right_hip = kp[11], kp[12]

                # Compute average ankle height
                avg_ankle_y = (left_ankle[1] + right_ankle[1]) / 2
                avg_knee_y = (left_knee[1] + right_knee[1]) / 2
                avg_hip_y = (left_hip[1] + right_hip[1]) / 2

                # Retrieve previous ankle position for this person
                prev_ankle_y = previous_ankle_heights.get(person_id, avg_ankle_y)

                # Detect jump: If ankle height has significantly increased (gone up)
                if prev_ankle_y - avg_ankle_y > jump_threshold:
                    action = "Jumping"
                    color = (0, 0, 255)  # Red
                else:
                    action = "Standing"
                    color = (0, 255, 0)  # Green

                # Save the current ankle height for next frame comparison
                previous_ankle_heights[person_id] = avg_ankle_y

                # Draw bounding box and label
                x1, y1, x2, y2 = map(int, result.boxes[person_id].xyxy[0])
                cv2.rectangle(frame, (x1, y1), (x2, y2), color, 2)
                cv2.putText(frame, action, (x1, y1 - 10),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.6, color, 2)

    # Write processed frame to video
    video_writer.write(frame)

# Release resources
cap.release()
video_writer.release()
print(f"✅ Jumping detection completed. Output saved at: {output_path}")